In [1]:
import numpy
import pandas as pd
import plotly.figure_factory as ff
import plotly.express as px

In [2]:
import dash

/Users/mostafa.shahhosseini/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
data1 = pd.read_csv("../data/logs_05-23_19-11-48.csv")
data2 = pd.read_csv("../data/logs_05-23_19-12-15.csv")
data3 = pd.read_csv("../data/logs_05-23_19-12-35.csv")
data4 = pd.read_csv("../data/logs_05-23_19-13-23.csv")

In [4]:
df = data2

In [5]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import plotly.graph_objects as go
import plotly.figure_factory as ff


# Assuming the dataframe df is already defined
labels = df['reaction'].unique()
counts = df['reaction'].value_counts().values.tolist()
df['time_label'] = pd.to_datetime(df.timestamp, unit = 'ms')
df['timestamp'] = pd.to_datetime(df.timestamp, unit = 'ms')
df.set_index('time_label', inplace=True)

# Initialize the Dash app
app = dash.Dash(__name__)

# Define the app layout
app.layout = html.Div([
    dcc.Dropdown(
        id='plot-type-dropdown',
        options=[{'label': 'Pie Chart', 'value': 'pie'},
                 {'label': 'Bar Plot', 'value': 'bar'},
                 {'label': 'Hourly Distribution of Interaction', 'value': 'hourly_dist'},
                 {'label': 'Change of Number of Reactions', 'value': 'reactions_along_time'},
                 {'label': 'Score Box Plot', 'value': 'total_score_box'},
                 {'label': 'Correlation of Press times and Reaction', 'value': 'press_action_correlation'},
                 {'label': "User's belief Change", 'value': 'user_change'},
                 {'label': "Reaction's Frequency Change", 'value': 'reaction_frequency_change'},
                 ],
        value='pie'
    ),
    dcc.Graph(id='displayed-plot'),
])

# Define the callback to update the displayed-plot
@app.callback(
    Output('displayed-plot', 'figure'),
    [Input('plot-type-dropdown', 'value')]
)
def update_graph(plot_type):
    emoji_mapping = {
            '🙁': 1,
            '👎': 2,
            '🙄': 3,
            '🙂': 4,
            '😆': 5,
            '👍': 6,
            '👏': 7,
            '😍': 8,
        }
    if plot_type == 'pie':
        fig = go.Figure(data=[go.Pie(labels=labels, values=counts)])
        fig.update_layout(title_text='Pie Chart of Reactions')
        return fig

    elif plot_type == 'bar':
        user_reaction_counts = df['userId'].value_counts()
        fig = go.Figure(data=[go.Bar(y=user_reaction_counts.values,
                                      hovertext=user_reaction_counts.index.astype(str))])
        fig.update_layout(title_text='Bar Plot of User Reactions', yaxis_title="Reaction Counts")
        return fig
    elif plot_type=="hourly_dist":
        df.index = pd.to_datetime(df.index)
        # Create an 'hour' column
        df['hour'] = df.index.hour
        hist_data = []
        group_labels = []
        # Get all unique reactions
        unique_reactions = df['reaction'].unique()
        # Loop through all unique reactions
        for reaction in unique_reactions:
            # get the hours where the reaction occurred
            reaction_hours = df[df['reaction'] == reaction].hour
            # Only add to plot if there are at least two unique values
            if len(reaction_hours.unique()) > 1:
                hist_data.append(reaction_hours)
                group_labels.append(reaction)

        # Create distplot with curve_type set to 'normal'
        fig = ff.create_distplot(hist_data, group_labels, bin_size=.2, curve_type='kde', show_hist=False,)
        fig.update_layout(title_text='Hourly Distribution for All Reactions')
        return fig
    elif plot_type=="reactions_along_time":
        data_count = df.groupby(df['timestamp'].dt.date).count()
        fig = go.Figure()
        fig.add_trace(go.Scatter(x=data_count.index, y=data_count['reaction'], mode='lines'))

        fig.update_layout(title_text='Number of Reactions Over Time (daily)',
                        xaxis_title='Date',
                        yaxis_title='Number of Reactions')
        return fig
    elif plot_type == "total_score_box":
        df['reaction_value'] = df['reaction'].map(emoji_mapping).fillna(0)
        fig = go.Figure()
        fig.add_trace(go.Box(x=df['reaction_value'], name='Reactions', orientation='h'))
        fig.update_layout(
            title='Distribution of Reactions',
            xaxis=dict(title='Reaction Value')
        )
        return fig
    elif plot_type == "press_action_correlation":
        df['reaction_numeric'] = df['reaction'].map(emoji_mapping)
        # Calculate the average number of presses per reaction for each user
        average_presses = df.groupby(['userId', 'reaction_numeric']).size().groupby('userId').mean().reset_index(name='average_presses')
        # Merge this information back into the dataframe
        df_ = pd.merge(df, average_presses, how='left', on='userId')
        # Create hexbin plot
        fig = go.Figure(go.Histogram2d(
            x=df_['reaction_numeric'],
            y=df_['average_presses'],
            autobinx=False,
            xbins=dict(start=0, end=8, size=1),  # Adjust size to change the resolution of the hexbins
            autobiny=False,
            ybins=dict(start=0, end=df_['average_presses'].max(), size=df_['average_presses'].max()/20),  # Adjust size to change the resolution of the hexbins
            colorscale='Viridis'
        ))
        fig.update_layout(
            title='Hexbin Plot of Average Button Presses vs Reaction Numeric Value',
            xaxis_title='Reaction Numeric Value',
            yaxis_title='Average Button Presses',
            autosize=False,
            width=600,
            height=600,
        )
        return fig
    elif plot_type=="user_change":
        user_id = 'ASWaWCYrTh3RpzUfABFP'
        user_data = df[df['userId'] == user_id]
        # Apply the mapping function to the reaction column
        user_data['reaction_value'] = user_data['reaction'].map(emoji_mapping).fillna(0)
        # Plot the change in reaction over time using Plotly
        fig = px.line(user_data, x='timestamp', y='reaction_value', title=f'Reaction Change Over Time for User ID: {user_id}')
        fig.update_xaxes(title_text='Timestamp')
        fig.update_yaxes(title_text='Reaction Value')
        return fig
    elif plot_type=="reaction_frequency_change":
        weekly_counts = df.resample('W').reaction.value_counts()
        # unstack the hierarchical index produced by value_counts
        weekly_counts = weekly_counts.unstack(level=-1, fill_value=0)
        fig = go.Figure()
        # Loop through each unique reaction
        for reaction in weekly_counts.columns:
            fig.add_trace(go.Scatter(x=weekly_counts.index, 
                                    y=weekly_counts[reaction], 
                                    mode='lines', 
                                    name=reaction))
        fig.update_layout(title='Reaction frequency over time',
                        xaxis_title='Time',
                        yaxis_title='Frequency')

        return fig
if __name__ == '__main__':
    app.run_server(debug=True)


/var/folders/g3/fv349bz92qn_mgn345hxy0k5wdvccj/T/ipykernel_30310/583630590.py:2: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc
/var/folders/g3/fv349bz92qn_mgn345hxy0k5wdvccj/T/ipykernel_30310/583630590.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
